# Dlib vs MediaPipe Face Mesh Comparison 

A comparison between the Dlib and MediaPipe Face Mesh comparison in order to determine which extraction method is more efficient for training and real time pain detection

In [ ]:
# Cell 2: imports
import os
import time
import numpy as np
import pandas as pd
from IPython.display import display

# Refactored function imports (must be in PYTHONPATH)
from data_preparation.processing_pipeline_dlib import init_dlib, video_to_landmark_vectors as dlib_video_to_vectors
from data_preparation.processing_pipeline_mediapipe import load_reference_keypoints, video_to_feature_sequences as mp_video_to_feature_sequences

print("Imports ready.")

In [ ]:
COLAB_ROOT = '/content/drive/MyDrive/PainRecognitionProject/'
DATA_DIR = os.path.join(COLAB_ROOT, 'data/BioVid_HeatPain/')
FRAME_SKIP_VALUE = 3  # adjust for profiling

# Dlib resource paths (edit if needed)
PREDICTOR_PATH = os.path.join(COLAB_ROOT, 'data', 'shape_predictor_68_face_landmarks.dat')
MEAN_FACE_PATH = os.path.join(COLAB_ROOT, 'data', 'landmarks_mean_face.npy')
WEIGHTS_PATH = os.path.join(COLAB_ROOT, 'data', 'frontalization_weights.npy')

def list_sample_videos(base_dir, limit=20):
    samples = []
    for subj in sorted(os.listdir(base_dir)):
        subj_path = os.path.join(base_dir, subj)
        if not os.path.isdir(subj_path):
            continue
        for fn in sorted(os.listdir(subj_path)):
            if fn.lower().endswith('.mp4'):
                samples.append(os.path.join(subj_path, fn))
                if len(samples) >= limit:
                    return samples
    return samples

sample_videos = list_sample_videos(DATA_DIR, limit=50)
print("Found sample videos:", len(sample_videos))
VIDEO_SAMPLE_PATH = sample_videos[0] if sample_videos else None
print("Default sample:", VIDEO_SAMPLE_PATH)

In [ ]:
dlib_ctx = init_dlib(PREDICTOR_PATH, MEAN_FACE_PATH, WEIGHTS_PATH)
print("Dlib initialized. Frontalization weights loaded:", dlib_ctx.get('frontalization_weights') is not None)

mp_ref_kp, mp_ref_ok = load_reference_keypoints(os.path.join(COLAB_ROOT, 'data', 'key_points_xyz.npy'))
print("MediaPipe reference loaded:", mp_ref_ok)

In [ ]:
def dlib_extractor_wrapper(video_path, frame_skip, visualize=False):
    return dlib_video_to_vectors(
        video_path=video_path,
        detector=dlib_ctx['detector'],
        predictor=dlib_ctx['predictor'],
        aligner=dlib_ctx.get('aligner'),
        frontalization_weights=dlib_ctx.get('frontalization_weights'),
        canonical_reference=dlib_ctx.get('canonical_reference'),
        frame_skip=frame_skip,
        frontalize=True,
        visualize=visualize
    )

def mp_extractor_wrapper(video_path, frame_skip, visualize=False):
    return mp_video_to_feature_sequences(
        video_path=video_path,
        frame_skip=frame_skip,
        reference_keypoints_3d=mp_ref_kp if mp_ref_ok else None,
        use_frontalization=bool(mp_ref_ok),
        visualize=visualize
    )

In [ ]:
def run_comparison(method_name, extraction_function, video_path, skip):
    if extraction_function is None:
        raise RuntimeError(f"Extractor for {method_name} is not available.")
    print(f"\n--- Testing: {method_name} (frame_skip={skip}) ---")
    start = time.time()
    seq = extraction_function(video_path=video_path, frame_skip=skip, visualize=False)
    elapsed = time.time() - start

    if not seq:
        n_frames = 0; vector_len = 0; vector_kb = 0.0
        print("No feature vectors extracted.")
    else:
        arr = np.array(seq)
        n_frames = arr.shape[0]
        vector_len = arr.shape[1] if arr.ndim >= 2 else (arr[0].shape[0] if n_frames>0 else 0)
        vector_kb = (arr[0].nbytes if n_frames>0 else 0) / 1024.0
        print(f" - Shape (N,features): {arr.shape}")

    fps = n_frames / elapsed if elapsed > 0 else 0.0
    print(f" - Time: {elapsed:.2f}s | FPS: {fps:.2f} | Frames: {n_frames}")
    print(f" - Vector length: {vector_len} | Vector size: {vector_kb:.3f} KB")

    return {'method': method_name, 'time_s': elapsed, 'fps': fps, 'n_frames': n_frames, 'vector_len': vector_len, 'vector_kb': vector_kb}

In [ ]:
# Cell 7: run single-sample benchmark
if VIDEO_SAMPLE_PATH is None:
    raise FileNotFoundError("No sample video found. Update VIDEO_SAMPLE_PATH or check DATA_DIR.")

results = []
results.append(run_comparison("Dlib (68 pts, 2D + Frontalization)", dlib_extractor_wrapper, VIDEO_SAMPLE_PATH, FRAME_SKIP_VALUE))
results.append(run_comparison("MediaPipe (478 pts, 3D + Frontalization)", mp_extractor_wrapper, VIDEO_SAMPLE_PATH, FRAME_SKIP_VALUE))

df_results = pd.DataFrame(results).set_index('method')
display(df_results.sort_values(by='fps', ascending=False))

In [ ]:
# Cell 8: multi-sample benchmark (averaging)
SAMPLES_TO_TEST = sample_videos[:10]  # change as needed
REPEAT_PER_SAMPLE = 1  # increase for better averaging
aggregate = []

for vid in SAMPLES_TO_TEST:
    for _ in range(REPEAT_PER_SAMPLE):
        try:
            aggregate.append(run_comparison("Dlib (68 pts, 2D + Frontalization)", dlib_extractor_wrapper, vid, FRAME_SKIP_VALUE))
            aggregate.append(run_comparison("MediaPipe (478 pts, 3D + Frontalization)", mp_extractor_wrapper, vid, FRAME_SKIP_VALUE))
        except Exception as e:
            print("Run error:", e)

if aggregate:
    df = pd.DataFrame(aggregate)
    summary = df.groupby('method').agg({
        'time_s': ['mean','std'],
        'fps': ['mean','std'],
        'n_frames': 'mean',
        'vector_len': 'mean',
        'vector_kb': ['mean','std']
    })
    display(summary)
else:
    print("No data collected.")

In [ ]:
# Cell 9: visual check (run manually)
do_visual_check = True  # set to True to visualize
if do_visual_check:
    print("Visualizing Dlib:")
    dlib_extractor_wrapper(VIDEO_SAMPLE_PATH, FRAME_SKIP_VALUE, visualize=True)
    print("Visualizing MediaPipe:")
    mp_extractor_wrapper(VIDEO_SAMPLE_PATH, FRAME_SKIP_VALUE, visualize=True)
else:
    print("Visual check disabled. Set do_visual_check = True and re-run.")